In [ ]:
import pandas as pd
import numpy as np
import kfold_and_metrics as km

from sklearn.metrics import roc_auc_score
import tensorflow as tf

tf.keras.backend.clear_session()

In [ ]:
df = pd.read_csv('Lero/final.csv')
tf.random.set_seed(22)

In [ ]:
def best_nn_params(df):

    activation_possibilities = ["softmax", "relu", "sigmoid"]
    max = 0
    best = []

    for activation_hidden in activation_possibilities:
        for num_nodes_hidden_layer in range(5, len(df.columns), 10):
            for gradient in [0.001, 0.003, 0.005, 0.007, 0.01]:

                nn_model = tf.keras.models.Sequential([
                tf.keras.layers.Flatten(input_shape=(len(df.columns))),
                tf.keras.layers.Dense(num_nodes_hidden_layer,activation=activation_hidden),
                tf.keras.layers.Dense(2,activation='softmax')
                ])

                nn_model.compile(
                    optimizer=tf.keras.optimizers.SGD(gradient), 
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
                    )

                score = km.k_fold_cv_keras(compiled_model=nn_model, df=df)
                avg, std = km.weighted_avg_and_std(np.array(score["roc_auc_score"]), score["weight"])
                
                if score > max:
                    best = [num_nodes_hidden_layer, activation_hidden, gradient]

    nn_model = tf.keras.models.Sequential([
                    tf.keras.layers.Flatten(input_shape=(len(df.columns))),
                    tf.keras.layers.Dense(best[0],activation=best[1]),
                    tf.keras.layers.Dense(2,activation='softmax')
                    ])
                
    nn_model.compile(
        optimizer=tf.keras.optimizers.SGD(best[2]), 
        loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
        )

    return nn_model
